In [1]:
kernel.silent(true)

In [2]:

import $ivy.`org.apache.spark::spark-mllib:2.4.5`

import $ivy.`org.apache.spark::spark-sql:2.4.5`


import org.apache.spark.ml.linalg.{Matrix, Vectors}
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import scala.collection.mutable.ListBuffer

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = NotebookSparkSession
    .builder()
    .master("local[*]")
    .getOrCreate()


Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

In [3]:
// Рисование
import $ivy.`org.plotly-scala::plotly-almond:0.7.2`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._
import $ivy.`org.scalanlp::breeze:0.13.2`
import $ivy.`org.scalanlp::breeze-natives:0.13.2`
import $ivy.`org.scalanlp::breeze-viz:0.13.2`

import breeze.linalg.DenseVector
import breeze.linalg.DenseMatrix
implicit def fromDenseVectorDouble(v: DenseVector[Double]): Sequence = v.toArray.toVector

In [4]:
import spark.implicits._

In [5]:
def getCorrDataFrame(df: DataFrame, replace_val: Double = 0.0, mode: String = "pearson"):DataFrame={
    // Получим имена колонок и типы для фильтрации
    var types = df.schema.fields.map(f=>f.dataType).toSeq.map(_.toString).to[ListBuffer]
    var names = df.schema.fields.map(f=>f.name).toSeq.map(_.toString).to[ListBuffer]
    var removeList:ListBuffer[String] = ListBuffer()
    // Пометим нечисловые и Id 
    for (i <-0 to types.size-1){
    if (((types(i)!= "IntegerType") & (types(i)!= "DoubleType"))|names(i).contains("Id")){
        removeList+=names(i)
        }
    
    }
    names--=removeList
    // Получим эффективно матрицу корреляции
    val assembler = new VectorAssembler()
      .setInputCols(names.toArray)
      .setOutputCol("features")
    var df_filled = df.na.fill(replace_val)
    val output = assembler.setHandleInvalid("skip").transform(df_filled).select("features")
    val matrix = Correlation.corr(output, "features", mode)
    var lm = matrix.head().getAs[Matrix](0)
    // конвертим в датафрейм массивов
    val cols = (0 until lm.numCols).toSeq

    val matrixRows = lm.rowIter.toSeq.map(_.toArray)
    val df_corr = spark.sparkContext.parallelize(matrixRows).toDF("Row")
    var df_corr_cols = cols.foldLeft(df_corr)(
            (df_corr, i) => df_corr.withColumn(names(i), $"Row".getItem(i))
            ).drop("arr")
    // Добавляем колонку
    val rdd_names = spark.sparkContext.parallelize(names)
    val rdd_new = df_corr_cols.rdd.zip(rdd_names).map(r => Row.fromSeq(r._1.toSeq ++ Seq(r._2)))

    spark.createDataFrame(rdd_new,  df_corr_cols.schema.add("ids", StringType))
    
}

In [6]:
var df = spark.read.parquet("../data/main_dataset/collabTrain")

parquet at cmd5.sc:1

48 / 48

parquet at cmd5.sc:1

1 / 1

In [7]:
df.printSchema

root
 |-- instanceId_userId: integer (nullable = true)
 |-- instanceId_objectType: string (nullable = true)
 |-- instanceId_objectId: integer (nullable = true)
 |-- audit_pos: long (nullable = true)
 |-- audit_clientType: string (nullable = true)
 |-- audit_timestamp: long (nullable = true)
 |-- audit_timePassed: long (nullable = true)
 |-- audit_experiment: string (nullable = true)
 |-- audit_resourceType: long (nullable = true)
 |-- metadata_ownerId: integer (nullable = true)
 |-- metadata_ownerType: string (nullable = true)
 |-- metadata_createdAt: long (nullable = true)
 |-- metadata_authorId: integer (nullable = true)
 |-- metadata_applicationId: long (nullable = true)
 |-- metadata_numCompanions: integer (nullable = true)
 |-- metadata_numPhotos: integer (nullable = true)
 |-- metadata_numPolls: integer (nullable = true)
 |-- metadata_numSymbols: integer (nullable = true)
 |-- metadata_numTokens: integer (nullable = true)
 |-- metadata_numVideos: integer (nullable = true)
 |-- me

 |-- auditweights_userOwner_USER_INTERNAL_LIKE: double (nullable = true)
 |-- auditweights_userOwner_USER_INTERNAL_UNLIKE: double (nullable = true)
 |-- auditweights_userOwner_USER_PRESENT_SEND: double (nullable = true)
 |-- auditweights_userOwner_USER_PROFILE_VIEW: double (nullable = true)
 |-- auditweights_userOwner_USER_SEND_MESSAGE: double (nullable = true)
 |-- auditweights_userOwner_USER_STATUS_COMMENT_CREATE: double (nullable = true)
 |-- auditweights_userOwner_VIDEO: double (nullable = true)
 |-- auditweights_userOwner_VOTE_POLL: double (nullable = true)
 |-- auditweights_x_ActorsRelations: long (nullable = true)
 |-- auditweights_likersSvd_spark_hyper: double (nullable = true)
 |-- auditweights_source_PROMO: double (nullable = true)
 |-- date: date (nullable = true)



In [8]:
println("owner types: ")
df.select("metadata_ownerType").distinct().show()
// нашли группы

owner types: 


show at cmd7.sc:2

28 / 28

show at cmd7.sc:2

1 / 1

...

show at cmd7.sc:2

100 / 100

show at cmd7.sc:2

75 / 75

+-------------------+
| metadata_ownerType|
+-------------------+
|GROUP_OPEN_OFFICIAL|
|         GROUP_OPEN|
+-------------------+



In [9]:
import org.apache.spark.sql.functions._


In [10]:
df.groupBy("membership_status").agg(countDistinct( $"instanceId_userId")).show()

show at cmd9.sc:1

28 / 28

show at cmd9.sc:1

200 / 200

show at cmd9.sc:1

...

show at cmd9.sc:1

100 / 100

show at cmd9.sc:1

75 / 75

+-----------------+---------------------------------+
|membership_status|count(DISTINCT instanceId_userId)|
+-----------------+---------------------------------+
|                !|                             1056|
|             null|                          2516907|
|                B|                              334|
|                Y|                             3251|
|                M|                             2418|
|                A|                          2772341|
|                R|                                3|
|                I|                             5509|
|                P|                            51214|
+-----------------+---------------------------------+



In [10]:
//Популярность групп - количество уникальных человек в группе - не подходит

// Потому что нельзя проинтерпретировать членство

In [11]:
// значит нужно упорядочить по числу залайканнх постов
// Рассмотрим все возможные комбинации - их много
df.groupBy($"feedback").agg(count( $"instanceId_userId").alias("countUser")).orderBy(desc("countUser")).show()

show at cmd10.sc:1

28 / 28

show at cmd10.sc:1

200 / 200

+--------------------+---------+
|            feedback|countUser|
+--------------------+---------+
|           [Ignored]| 11633708|
|             [Liked]|  2445356|
|           [Clicked]|  1308199|
|          [Disliked]|  1174102|
| [Disliked, Ignored]|   408852|
|    [Clicked, Liked]|   396484|
|  [Clicked, Ignored]|   165655|
|    [Liked, Ignored]|   120053|
|[Clicked, ReShare...|    90873|
|   [Clicked, Viewed]|    76645|
| [Clicked, ReShared]|    70155|
|            [Viewed]|    59587|
|   [ReShared, Liked]|    52471|
|          [ReShared]|    48160|
| [Disliked, Clicked]|    30917|
|[Clicked, Unliked...|    23978|
|[Clicked, ReShare...|    19502|
|   [Disliked, Liked]|    18621|
|         [Commented]|    15488|
| [ReShared, Ignored]|    13463|
+--------------------+---------+
only showing top 20 rows



In [12]:
// Поймем сколько уникальных состояний контента мб и сделаем pivot
val distinct_df = df.withColumn("feedback_explodes", explode($"feedback")).
                     agg(collect_set("feedback_explodes").alias("distinct_feat"))

In [13]:
val u_name = distinct_df.select("distinct_feat").rdd//.map(_.toSeq).flatten
distinct_df.select("distinct_feat").show(false)

show at cmd12.sc:2

28 / 28

show at cmd12.sc:2

1 / 1

+------------------------------------------------------------------------------------+
|distinct_feat                                                                       |
+------------------------------------------------------------------------------------+
|[Liked, Ignored, ReShared, Viewed, Complaint, Unliked, Commented, Clicked, Disliked]|
+------------------------------------------------------------------------------------+



In [14]:
val new_list = List("Liked", "Ignored", "ReShared", "Viewed", "Complaint", "Unliked", "Commented", "Clicked", "Disliked")

In [15]:
// Добавим  1 и 0 для каждой реакции
for (i <- new_list){
    df = df.withColumn(i, when(array_contains($"feedback", i ), 1.0).otherwise(0.0))
}

In [16]:
// Сколько лайков поставили всего
df.select("Liked").agg(sum($"Liked")).show()

show at cmd15.sc:1

28 / 28

show at cmd15.sc:1

1 / 1

+----------+
|sum(Liked)|
+----------+
| 3220429.0|
+----------+



In [17]:
//  Выведем топ 20 по популярности групп
df.groupBy("metadata_ownerId").agg(sum("Liked").alias("popularity")).orderBy(desc("popularity")).show()

show at cmd16.sc:1

28 / 28

show at cmd16.sc:1

200 / 200

+----------------+----------+
|metadata_ownerId|popularity|
+----------------+----------+
|           37463|   16726.0|
|           11222|   12301.0|
|           18942|   10160.0|
|           14285|   10024.0|
|           62833|    9428.0|
|           26843|    8779.0|
|           19481|    8717.0|
|           79619|    8519.0|
|           11058|    8384.0|
|           26735|    7770.0|
|           77398|    7628.0|
|           11163|    7495.0|
|            9182|    7304.0|
|           60195|    7270.0|
|           64184|    7216.0|
|           76851|    7153.0|
|           14959|    7057.0|
|           65305|    6953.0|
|           38703|    6900.0|
|           17215|    6663.0|
+----------------+----------+
only showing top 20 rows



In [18]:
val distinct_df = df.withColumn("meta_explodes", explode($"metadata_options")).
                     agg(collect_set("meta_explodes").alias("distinct_meta"))

In [19]:
// Посмотрим на всякий случай метадату - кажется, из нее можно извлечь множество табличных признаков по аналогии с фидбеками
distinct_df.select("distinct_meta").show(false)

show at cmd18.sc:1

28 / 28

show at cmd18.sc:1

1 / 1

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|distinct_meta                                                                                                                                                                                                                                                                     |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[IS_GIF, IS_CENSORED, IS_PART_OF_ALBUM, IS_PART_OF_TOPIC, HAS_URLS, HAS_POLLS, IS_INTERNAL_SHARE, IS_INTERNAL_GROUP_SHARE, HAS_PINS, HAS_DETECTED_TEXT, IS_PROMO, HAS_MU

In [20]:

var meta_list = distinct_df.collect.map(_.getSeq[String](0).toArray).flatten

collect at cmd19.sc:1

28 / 28

collect at cmd19.sc:1

1 / 1

In [21]:
// Добавим  1 и 0 для каждой реакции
for (i <- meta_list){
    df = df.withColumn(i, when(array_contains($"metadata_options", i ), 1.0).otherwise(0.0))
}

In [22]:
df.printSchema

root
 |-- instanceId_userId: integer (nullable = true)
 |-- instanceId_objectType: string (nullable = true)
 |-- instanceId_objectId: integer (nullable = true)
 |-- audit_pos: long (nullable = true)
 |-- audit_clientType: string (nullable = true)
 |-- audit_timestamp: long (nullable = true)
 |-- audit_timePassed: long (nullable = true)
 |-- audit_experiment: string (nullable = true)
 |-- audit_resourceType: long (nullable = true)
 |-- metadata_ownerId: integer (nullable = true)
 |-- metadata_ownerType: string (nullable = true)
 |-- metadata_createdAt: long (nullable = true)
 |-- metadata_authorId: integer (nullable = true)
 |-- metadata_applicationId: long (nullable = true)
 |-- metadata_numCompanions: integer (nullable = true)
 |-- metadata_numPhotos: integer (nullable = true)
 |-- metadata_numPolls: integer (nullable = true)
 |-- metadata_numSymbols: integer (nullable = true)
 |-- metadata_numTokens: integer (nullable = true)
 |-- metadata_numVideos: integer (nullable = true)
 |-- me

 |-- auditweights_userOwner_USER_INTERNAL_LIKE: double (nullable = true)
 |-- auditweights_userOwner_USER_INTERNAL_UNLIKE: double (nullable = true)
 |-- auditweights_userOwner_USER_PRESENT_SEND: double (nullable = true)
 |-- auditweights_userOwner_USER_PROFILE_VIEW: double (nullable = true)
 |-- auditweights_userOwner_USER_SEND_MESSAGE: double (nullable = true)
 |-- auditweights_userOwner_USER_STATUS_COMMENT_CREATE: double (nullable = true)
 |-- auditweights_userOwner_VIDEO: double (nullable = true)
 |-- auditweights_userOwner_VOTE_POLL: double (nullable = true)
 |-- auditweights_x_ActorsRelations: long (nullable = true)
 |-- auditweights_likersSvd_spark_hyper: double (nullable = true)
 |-- auditweights_source_PROMO: double (nullable = true)
 |-- date: date (nullable = true)
 |-- Liked: double (nullable = false)
 |-- Ignored: double (nullable = false)
 |-- ReShared: double (nullable = false)
 |-- Viewed: double (nullable = false)
 |-- Complaint: double (nullable = false)
 |-- Unliked: 

In [23]:
// извлечем часы, преобразуем timestamp
df = (df
    .withColumn("PosTime", ($"audit_timestamp" / 1000).cast(TimestampType))
    .withColumn("hourPosTime", hour($"PosTime").cast(DoubleType))
      )

In [24]:
df.select("date", "PosTime", "hourPosTime").show(false)

show at cmd23.sc:1

1 / 1

+----------+-----------------------+-----------+
|date      |PosTime                |hourPosTime|
+----------+-----------------------+-----------+
|2018-03-04|2018-03-04 20:08:06.477|20.0       |
|2018-03-04|2018-03-03 21:38:46.655|21.0       |
|2018-03-04|2018-03-04 16:33:07.538|16.0       |
|2018-03-04|2018-03-04 07:42:19.921|7.0        |
|2018-03-04|2018-03-04 16:23:15.763|16.0       |
|2018-03-04|2018-03-04 10:46:51.706|10.0       |
|2018-03-04|2018-03-04 07:42:35.325|7.0        |
|2018-03-04|2018-03-04 07:42:35.325|7.0        |
|2018-03-04|2018-03-04 10:24:15.559|10.0       |
|2018-03-04|2018-03-04 16:24:08.026|16.0       |
|2018-03-04|2018-03-04 15:29:30.539|15.0       |
|2018-03-04|2018-03-04 14:38:06.31 |14.0       |
|2018-03-04|2018-03-04 04:14:40.719|4.0        |
|2018-03-04|2018-03-04 17:16:05.245|17.0       |
|2018-03-04|2018-03-04 05:35:22.673|5.0        |
|2018-03-04|2018-03-04 15:50:26.509|15.0       |
|2018-03-04|2018-03-04 16:19:21.924|16.0       |
|2018-03-04|2018-03-

In [25]:
val res = df.groupBy("hourPosTime").agg(sum("Liked").alias("Liked")).rdd

In [26]:
// Соберем количество лайков в каждый час
val arr_res = res.map(_.toSeq.toArray).map(_.map(_.toString.toDouble)).collect//.map(_.toString.toDouble)


rdd at cmd24.sc:1

28 / 28

collect at cmd25.sc:1

200 / 200

In [27]:
val matrix = DenseMatrix(arr_res:_*)

In [28]:
val trace_hist = Bar(matrix(::,0), matrix(::,1))
trace_hist.plot(title ="Количество лайков от часа в сутках")


Выводы по гистограмме
<ul>
    <li>Похоже, что нам нужно +3 часа прибавить и исходное время московске</li>
    <li>Пиковые часы - после работы</li>
    <li>Ночью лайки ставят меньше, потому что и пользователей меньше</li>
    <li>Лайки ставят все время, когда бодрствует большинство</li>
    </ul>

In [29]:
var df_temp = getCorrDataFrame(df)

first at RowMatrix.scala:61

1 / 1

treeAggregate at RowMatrix.scala:433

28 / 28

treeAggregate at RowMatrix.scala:433

...

treeAggregate at RowMatrix.scala:122

28 / 28

treeAggregate at RowMatrix.scala:122

4 / 4

20/11/17 01:06:59 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
20/11/17 01:06:59 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [30]:
// Рассчитаем корреляцию, хоть у нас и не континуальная переменная
(df_temp
.filter(!($"ids".isin("Liked")))
.select("ids", "Liked").na.drop()
.withColumn("abs_like", abs($"Liked"))
.orderBy(desc("abs_like"))
.select("ids", "Liked")
.show(148,false)
 )

show at cmd29.sc:7

8 / 8

+----------------------------------------+----------------------+
|ids                                     |Liked                 |
+----------------------------------------+----------------------+
|Ignored                                 |-0.6259039397446567   |
|auditweights_ctr_high                   |0.14548656923887704   |
|Disliked                                |-0.13006402694750466  |
|ReShared                                |0.11756336473525891   |
|auditweights_userOwner_CREATE_LIKE      |0.0930743514886664    |
|Unliked                                 |0.09026606680206518   |
|HAS_PHOTOS                              |0.08968884427170941   |
|auditweights_isRandom                   |0.0893591324748627    |
|auditweights_svd_prelaunch              |0.08727070887180428   |
|HAS_POLLS                               |0.08324690747359784   |
|metadata_numPolls                       |0.08114650487156008   |
|userOwnerCounters_CREATE_LIKE           |0.08072226903427521   |
|auditweig

In [31]:
// Используем вместо этого roc_auc, как замену корреляции
// Для признака с самой высокой корреляцией, не являющегося стейтом
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.regression.LabeledPoint

In [32]:
df = df.na.fill(0)
val predictionAndLabels = df.select($"auditweights_ctr_high", $"Liked").rdd.map(x=>(x.getDouble(0), x.getDouble(1)))//.map { case LabeledPoint(label, features) =>
  //(rddX, rddY)
//}

In [33]:
var metrics = new BinaryClassificationMetrics(predictionAndLabels)

var auROC = metrics.areaUnderROC
println(s"Area under ROC = $auROC")

map at cmd31.sc:2

28 / 28

sortByKey at BinaryClassificationMetrics.scala:155

28 / 28

combineByKey at BinaryClassificationMetrics.scala:154

...

collect at SlidingRDD.scala:78

30 / 30

aggregate at AreaUnderCurve.scala:45

29 / 29

Area under ROC = 0.6394043711893063


In [34]:
val predictionAndLabels = df.select($"auditweights_userOwner_CREATE_LIKE", $"Liked").rdd.map(x=>(x.getDouble(0), x.getDouble(1)))
var metrics = new BinaryClassificationMetrics(predictionAndLabels)

var auROC = metrics.areaUnderROC
println(s"Area under ROC = $auROC")

map at cmd33.sc:1

28 / 28

sortByKey at BinaryClassificationMetrics.scala:155

28 / 28

combineByKey at BinaryClassificationMetrics.scala:154

...

collect at SlidingRDD.scala:78

30 / 30

aggregate at AreaUnderCurve.scala:45

29 / 29

Area under ROC = 0.5684241417191213


Вывод
<ul>
    <li>Топ коррелирующих признаков - Ignored, auditweights_ctr_high, Disliked. кажется, что можно проигнорировать, но поставить лайк в каких-то случаях, раз корреляция не -1.0. Что станно, ну да и ладно </li>
    <li>В  качестве альтернативы показателя корреляции был выбран roc auc, который более подходит характеру перменных, но рассчитывается много дольше</li>
    <li>Признаки состояния в топе по корреляции, но не являются взаимоисключающими, поэтому их интеракции лучше не рассматривать и предсказывать по отдельности</li>
    <li>Судя по корреляциям - лайк будет тяжеловато предсказать линейным моделям, возможно, стоит рассмотреть более нелинейные зависимости или нагенерировать большее количество признаков</li>
    </ul>

In [35]:
// Поисследуем оставшиеся реакции
(df_temp
.filter(!($"ids".isin("Disliked")))
.select("ids", "Disliked").na.drop()
.withColumn("abs_like", abs($"Disliked"))
.orderBy(desc("abs_like"))
.select("ids", "Disliked")
.show(148,false)
 )

show at cmd34.sc:7

8 / 8

+----------------------------------------+----------------------+
|ids                                     |Disliked              |
+----------------------------------------+----------------------+
|auditweights_isRandom                   |-0.6967912889122876   |
|userOwnerCounters_USER_FEED_REMOVE      |0.3923866436645584    |
|Ignored                                 |-0.28605758071082654  |
|auditweights_userOwner_USER_FEED_REMOVE |0.16976493341992013   |
|auditweights_userOwner_TEXT             |0.13491129128759438   |
|Liked                                   |-0.13006402694750466  |
|user_birth_date                         |-0.11298292369566783  |
|auditweights_userAge                    |0.11287331128348413   |
|auditweights_svd_spark                  |-0.11199583097006773  |
|auditweights_svd_prelaunch              |-0.10488271473780682  |
|user_gender                             |-0.09162573711821928  |
|auditweights_userOwner_IMAGE            |0.08954772814002501   |
|Clicked  

In [36]:
val predictionAndLabels = df.select($"userOwnerCounters_USER_FEED_REMOVE", $"Disliked").rdd.map(x=>(x.getDouble(0), x.getDouble(1)))
var metrics = new BinaryClassificationMetrics(predictionAndLabels)

var auROC = metrics.areaUnderROC
println(s"Area under ROC = $auROC")

map at cmd35.sc:1

28 / 28

sortByKey at BinaryClassificationMetrics.scala:155

28 / 28

combineByKey at BinaryClassificationMetrics.scala:154

...

collect at SlidingRDD.scala:78

30 / 30

aggregate at AreaUnderCurve.scala:45

29 / 29

Area under ROC = 0.9182316768875729


Вывод
<ul>
    <li>Топ коррелирующих признаков - auditweights_isRandom, userOwnerCounters_USER_FEED_REMOVE, Ignored. Дизлайки - более сильный сигнал, тогда как лайки ставятся всем подряд, выбешивает пользователя скорее что-то конкретное, что бесило его уже ни раз. </li>
    <li>Постановка дизлайка не взаимоисключает лайк, но при этом такие маркерные посты неплохо должны сегментировать аудиторию по контенту</li>
    </ul>

In [37]:
(df_temp
.filter(!($"ids".isin("Viewed")))
.select("ids", "Viewed").na.drop()
.withColumn("abs_like", abs($"Viewed"))
.orderBy(desc("abs_like"))
.select("ids", "Viewed")
.show(148,false)
 )

show at cmd36.sc:7

8 / 8

+----------------------------------------+----------------------+
|ids                                     |Viewed                |
+----------------------------------------+----------------------+
|IS_PART_OF_ALBUM                        |0.27975777751187164   |
|IS_PART_OF_TOPIC                        |-0.2546605705256014   |
|auditweights_notOriginalPhoto           |0.22185006917002925   |
|auditweights_userOwner_UNKNOWN          |0.2080199359871507    |
|HAS_DETECTED_TEXT                       |0.18544904914933244   |
|HAS_PHOTOS                              |-0.17265714804812768  |
|Clicked                                 |0.13755987765139271   |
|Ignored                                 |-0.12295476219106868  |
|IS_GIF                                  |0.10819004137063838   |
|auditweights_userOwner_IMAGE            |0.10448937223983049   |
|userOwnerCounters_IMAGE                 |0.096885494426535     |
|auditweights_userOwner_CREATE_IMAGE     |0.08810351155186018   |
|auditweig

In [38]:
val predictionAndLabels = df.select($"auditweights_notOriginalPhoto", $"Viewed").rdd.map(x=>(x.getDouble(0), x.getDouble(1)))
var metrics = new BinaryClassificationMetrics(predictionAndLabels)

var auROC = metrics.areaUnderROC
println(s"Area under ROC = $auROC")

map at cmd37.sc:1

28 / 28

sortByKey at BinaryClassificationMetrics.scala:155

28 / 28

combineByKey at BinaryClassificationMetrics.scala:154

...

collect at SlidingRDD.scala:78

4 / 4

aggregate at AreaUnderCurve.scala:45

3 / 3

Area under ROC = 0.7428428156828181


Вывод
<ul>
    <li>Топ коррелирующих признаков - auditweights_notOriginalPhoto, auditweights_userOwner_UNKNOWN, IS_PART_OF_ALBUM, IS_PART_OF_TOPIC. Подозреваю, что не оригинальное фото - это мемы и их смотрят чаще всего. Также люди интересуются незнакомцами. </li>
    <li>Кажется, чт реакции просмотра и дизлайка более обхяснимы текущими признаками</li>
    <li>Признаки состояния в топе по корреляции, но не являются взаимоисключающими, поэтому их интеракции лучше не рассматривать и предсказывать по отдельности</li>
    </ul>